# Notebook 1.3: Hierarchical Pattern

Este notebook prueba el patrón **DOCUMENT_SECTION_PARAGRAPH** con datos reales.

## Objetivos

1. **Limpiar el grafo** antes de empezar
2. **Crear patrón DOCUMENT_SECTION_PARAGRAPH** con estructura jerárquica
3. **Ingerir 3 documentos** usando el patrón jerárquico
4. **Explorar el grafo** creado
5. **Probar búsquedas** con los patrones GraphRAG implementados

## Patrón DOCUMENT_SECTION_PARAGRAPH

- **Document**: Documento completo
- **Section**: Secciones dentro del documento
- **Paragraph**: Párrafos dentro de las secciones

**Estructura:**
```
Document -[:HAS_SECTION]-> Section -[:HAS_PARAGRAPH]-> Paragraph
Paragraph -[:NEXT_PARAGRAPH]-> Paragraph
```

In [ ]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    in order to work on notebooks and scripts
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
                print(f"Path Folder parent added: {parent_dir}")
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
                print(f"Path Folder {path_folder} added: {candidate}")
            return
    print(f"Not found '{path_folder}' folder on the hierarchy of directories")

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")

In [ ]:
# Importar librerías necesarias
import sys
from pathlib import Path
from typing import List, Dict, Any

# Importar handlers
from src.utils.handlers import find_in_project

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

# Importar servicios para limpieza
from infrastructure.services.neo4j_index_service import Neo4jIndexService

# Importar patrones
from domain.value_objects.graph_pattern import GraphPattern, NodeDefinition, RelationshipDefinition

print(f"📦 Ungraph version: {ungraph.__version__}")

## Parte 1: Configuración y Limpieza

Configuramos Neo4j y limpiamos el grafo antes de empezar.

In [ ]:
# Configurar Neo4j
ungraph.configure(
    neo4j_uri="bolt://localhost:7687",
    neo4j_user="neo4j",
    neo4j_password="Ungraph22",  # ⚠️ CAMBIAR: Usa tu contraseña real
    neo4j_database="neo4j",
    embedding_model="sentence-transformers/all-MiniLM-L6-v2"
)

print("✅ Configuración completada")

In [ ]:
# Limpiar el grafo antes de empezar
print("🧹 Limpiando grafo...")
print("=" * 80)

index_service = Neo4jIndexService()

# Limpiar todos los nodos y relaciones
try:
    index_service.clean_graph()
    print("✅ Grafo limpiado (todos los nodos y relaciones eliminados)")
except Exception as e:
    print(f"⚠️  Error al limpiar grafo: {e}")

# Eliminar todos los índices
try:
    index_service.drop_all_indexes()
    print("✅ Índices eliminados")
except Exception as e:
    print(f"⚠️  Error al eliminar índices: {e}")

print("\n✅ Limpieza completada. Listo para ingesta.")

## Parte 2: Preparar Documentos

Localizamos los 3 documentos de prueba.

In [ ]:
# Encontrar carpeta de datos
data_path = find_in_project(
    target="data",
    search_type="folder",
    project_root=None
)

if data_path:
    print(f"✅ Carpeta de datos encontrada: {data_path}")
    
    # Seleccionar los 3 documentos de prueba
    test_files = [
        data_path / "110225.md",
        data_path / "AnnyLetter.txt",
        data_path / "Usar símboles de silencio de corchea.docx"
    ]
    
    # Verificar que existen
    available_files = [f for f in test_files if f.exists()]
    print(f"\n📄 Archivos disponibles ({len(available_files)}/{len(test_files)}):")
    for f in available_files:
        print(f"   ✅ {f.name}")
    
    for f in test_files:
        if not f.exists():
            print(f"   ⚠️  No encontrado: {f.name}")
else:
    print("❌ Carpeta de datos no encontrada")
    available_files = []

## Parte 3: Crear Patrón DOCUMENT_SECTION_PARAGRAPH

Definimos el patrón jerárquico con Document, Section y Paragraph.

In [ ]:
# Crear patrón DOCUMENT_SECTION_PARAGRAPH
print("📝 CREANDO PATRÓN DOCUMENT_SECTION_PARAGRAPH")
print("=" * 80)

# Nodo Document
document_node = NodeDefinition(
    label="Document",
    required_properties={"doc_id": str, "title": str},
    optional_properties={"created_at": int, "author": str},
    indexes=["doc_id"]
)

# Nodo Section
section_node = NodeDefinition(
    label="Section",
    required_properties={"section_id": str, "title": str, "order": int},
    optional_properties={"summary": str},
    indexes=["section_id"]
)

# Nodo Paragraph
paragraph_node = NodeDefinition(
    label="Paragraph",
    required_properties={
        "para_id": str,
        "content": str,
        "embeddings": list,
        "embeddings_dimensions": int
    },
    optional_properties={"order": int},
    indexes=["para_id"]
)

# Relaciones
has_section = RelationshipDefinition(
    from_node="Document",
    to_node="Section",
    relationship_type="HAS_SECTION",
    direction="OUTGOING"
)

has_paragraph = RelationshipDefinition(
    from_node="Section",
    to_node="Paragraph",
    relationship_type="HAS_PARAGRAPH",
    direction="OUTGOING"
)

next_paragraph = RelationshipDefinition(
    from_node="Paragraph",
    to_node="Paragraph",
    relationship_type="NEXT_PARAGRAPH",
    direction="OUTGOING"
)

DOCUMENT_SECTION_PARAGRAPH_PATTERN = GraphPattern(
    name="DOCUMENT_SECTION_PARAGRAPH",
    description="Patrón jerárquico: Document contiene Sections, Sections contienen Paragraphs.",
    node_definitions=[document_node, section_node, paragraph_node],
    relationship_definitions=[has_section, has_paragraph, next_paragraph],
    search_patterns=["basic", "hybrid", "parent_child"]
)

print(f"✅ Patrón creado: {DOCUMENT_SECTION_PARAGRAPH_PATTERN.name}")
print(f"   Nodos: {[n.label for n in DOCUMENT_SECTION_PARAGRAPH_PATTERN.node_definitions]}")
print(f"   Relaciones: {[r.relationship_type for r in DOCUMENT_SECTION_PARAGRAPH_PATTERN.relationship_definitions]}")

## Parte 4: Ingesta con Patrón DOCUMENT_SECTION_PARAGRAPH

Ingerimos los documentos usando el patrón jerárquico.

In [ ]:
# Ingesta con patrón DOCUMENT_SECTION_PARAGRAPH
print("📥 INGESTA CON PATRÓN DOCUMENT_SECTION_PARAGRAPH")
print("=" * 80)

all_chunks_hierarchical = []

for file_path in available_files:
    print(f"\n📄 Procesando: {file_path.name}")
    try:
        chunks = ungraph.ingest_document(
            file_path,
            pattern=DOCUMENT_SECTION_PARAGRAPH_PATTERN,
            chunk_size=1000,
            chunk_overlap=200,
            clean_text=True
        )
        all_chunks_hierarchical.extend(chunks)
        print(f"   ✅ {len(chunks)} chunks creados")
    except Exception as e:
        print(f"   ❌ Error: {e}")

print(f"\n✅ Total chunks con patrón DOCUMENT_SECTION_PARAGRAPH: {len(all_chunks_hierarchical)}")
print("\n💡 Nota: Este patrón requiere detección de secciones.")
print("   En una implementación completa, se usaría análisis de estructura del documento.")

## Parte 4: Explorar Grafo

Exploramos la estructura del grafo creado.

In [ ]:
# Explorar estructura del grafo
from src.utils.graph_operations import graph_session

driver = graph_session()
with driver.session() as session:
    # Contar nodos por tipo
    result = session.run("""
        MATCH (n)
        RETURN labels(n)[0] as label, count(n) as count
        ORDER BY count DESC
    """)
    
    print("📊 ESTRUCTURA DEL GRAFO:")
    print("=" * 80)
    for record in result:
        print(f"   {record['label']}: {record['count']} nodos")
    
    # Contar relaciones
    result = session.run("""
        MATCH ()-[r]->()
        RETURN type(r) as rel_type, count(r) as count
        ORDER BY count DESC
    """)
    
    print("\n🔗 RELACIONES:")
    for record in result:
        print(f"   {record['rel_type']}: {record['count']} relaciones")

driver.close()

## Parte 4.1: Visualizar Grafo

Visualizamos el grafo usando yFiles for Jupyter.

In [ ]:
# Importar funciones de visualización
from src.notebooks.graph_visualization import (
    visualize_file_page_chunk_pattern,
    visualize_simple_chunk_pattern,
    visualize_lexical_graph_pattern,
    visualize_hierarchical_pattern,
    visualize_sequential_chunks_pattern,
    visualize_pattern_structure,
    visualize_custom_query
)

print("✅ Funciones de visualización importadas")

## Parte 5: Probar Búsquedas

Probamos los patrones de búsqueda GraphRAG implementados.

In [ ]:
# Probar búsquedas
test_query = "test"
print(f"🔍 PROBANDO BÚSQUEDAS CON QUERY: '{test_query}'")
print("=" * 80)

# 1. Basic Retriever
print("\n1. Basic Retriever:")
try:
    results = ungraph.search_with_pattern(
        test_query,
        pattern_type="basic",
        limit=3
    )
    print(f"   ✅ {len(results)} resultados")
    if results:
        print(f"   Score promedio: {sum(r.score for r in results) / len(results):.3f}")
except Exception as e:
    print(f"   ❌ Error: {e}")

# 2. Metadata Filtering
print("\n2. Metadata Filtering:")
try:
    # Obtener un filename del grafo
    driver = graph_session()
    with driver.session() as session:
        result = session.run("MATCH (f:File) RETURN f.filename as filename LIMIT 1")
        record = result.single()
        if not record:
            # Intentar con Chunk si no hay File
            result = session.run("MATCH (c:Chunk) RETURN c.source_file as filename LIMIT 1")
            record = result.single()
        if record:
            filename = record["filename"]
            results = ungraph.search_with_pattern(
                test_query,
                pattern_type="metadata_filtering",
                metadata_filters={"filename": filename},
                limit=3
            )
            print(f"   ✅ {len(results)} resultados (filtrado por '{filename}')")
        else:
            print("   ⚠️  No hay archivos en el grafo")
    driver.close()
except Exception as e:
    print(f"   ❌ Error: {e}")

# 3. Parent-Child Retriever (si aplica)
print("\n3. Parent-Child Retriever:")
try:
    results = ungraph.search_with_pattern(
        test_query,
        pattern_type="parent_child",
        parent_label="Page",
        child_label="Chunk",
        relationship_type="HAS_CHUNK",
        limit=3
    )
    print(f"   ✅ {len(results)} resultados")
except Exception as e:
    print(f"   ⚠️  No aplica para este patrón: {e}")

In [ ]:
# Visualizar patrón DOCUMENT_SECTION_PARAGRAPH
print("🎨 VISUALIZANDO PATRÓN DOCUMENT_SECTION_PARAGRAPH")
print("=" * 80)

driver = graph_session()
try:
    visualize_hierarchical_pattern(driver, limit=12)
except Exception as e:
    print(f"⚠️  Error al visualizar: {e}")
    print("💡 Asegúrate de tener yfiles_jupyter_graphs_for_neo4j instalado")
finally:
    driver.close()

## Parte 6: Resumen

Resumen del patrón DOCUMENT_SECTION_PARAGRAPH.

In [ ]:
print("📊 RESUMEN DEL PATRÓN DOCUMENT_SECTION_PARAGRAPH")
print("=" * 80)
print(f"Chunks creados: {len(all_chunks_hierarchical)}")
print(f"Estructura: Document -[:HAS_SECTION]-> Section -[:HAS_PARAGRAPH]-> Paragraph")
print(f"Uso recomendado: Documentos con estructura jerárquica clara")
print("\n✅ Notebook completado exitosamente")